## July 22 2021

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [2]:
start_time = time.time()
df = pd.read_feather('D:/internship-telkom/data-science/slow-connection/CW02_dataset.ftr')
print('Time elapsed: {}'.format((time.time() - start_time)))

Time elapsed: 7.515970945358276


In [4]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.read_feather('D:/big-datasets/internship-telkom/CW02_dataset.ftr')

def make_df_bin(bin_1h, sample):
    cols = ['user_id', 'date_time', 'reportdate', 'bin_1h', 'int_latency_score', 'ext_latency_score', 'fd_latency_score',
            'int_ploss_score', 'ext_ploss_score', 'ses_setup_score', 'ser_acces_score']

    report1 = df[cols][(df['bin_1h'] >= bin_1h) & (df['bin_1h'] < 0)].drop_duplicates().sample(sample)
    report1.drop('user_id', axis=1, inplace=True)
    report1['report_0_1'] = 1
    
    report0 = df[cols][(df['bin_1h'] < bin_1h) | (df['bin_1h'] > 0)].drop_duplicates().sample(sample)
    report0.drop('user_id', axis=1, inplace=True)
    report0['report_0_1'] = 0
    
    df_report = pd.concat([report1, report0])
    return df_report

        
def make_df_score(score_name, bin_1h=-48, sample=100000):
    try:
        cols = ['name', 'score1', 'score2', 'score3', 'score4', 'scoremin1', 'report']
        score_lst = [1, 2, 3, 4, -1]

        df_report = make_df_bin(bin_1h, sample)
        df_score = pd.DataFrame(columns=cols)
        df_score['report'] = [0, 1]
        for n1, score in zip(range(1, 6), score_lst):
            for n2 in range(0, 2):
                df_score[cols[n1]][df_score['report'] == n2] = df_report[(df_report['report_0_1'] == n2) &
                                                                         (df_report[score_name] == score)].shape[0]                                                                  
        df_score['name'] = score_name
        return df_score
    except:
        print('Error! Please specify less smaller sample size.')


cols = [col for col in df.columns if 'score' in col]

df_conc = pd.concat([make_df_score(cols[0]), make_df_score(cols[1]), make_df_score(cols[2]), make_df_score(cols[3]),
                     make_df_score(cols[4]), make_df_score(cols[5]), make_df_score(cols[6])])
df_conc.reset_index(drop=True, inplace=True)
df_conc[['score1', 'score2', 'score3', 'score4', 'scoremin1']] = round(df_conc[['score1', 'score2', 'score3', 'score4', 'scoremin1']].astype(int) / 1000, 2)
print(df_conc)

                 name  score1  score2  score3  score4  scoremin1  report
0   int_latency_score   81.16    5.02    2.16    3.96       7.70       0
1   int_latency_score   75.63    5.46    2.36    4.49      12.06       1
2   ext_latency_score   89.06    2.31    0.26    0.27       8.09       0
3   ext_latency_score   84.37    2.02    0.25    0.30      13.05       1
4    fd_latency_score   43.43   26.71    9.32   14.78       5.76       0
5    fd_latency_score   39.53   24.77    9.05   15.98      10.66       1
6     int_ploss_score   70.85   25.91    1.48    1.55       0.21       0
7     int_ploss_score   66.52   26.66    1.98    2.22       2.63       1
8     ext_ploss_score   86.79   10.36    0.67    0.65       1.54       0
9     ext_ploss_score   82.98   11.22    0.94    2.60       2.26       1
10    ses_setup_score   89.32    8.03    1.59    0.88       0.17       0
11    ses_setup_score   88.63    8.04    1.97    1.05       0.31       1
12    ser_acces_score   19.77   49.97   21.45    8.